In [1]:
from modules.utils.database_utils import get_db_connection

get_db_connection()

<connection object at 0x1079813f0; dsn: 'user=app_user password=xxx channel_binding=require connect_timeout=10 dbname=neondb host=ep-tiny-union-abuvf65z-pooler.eu-west-2.aws.neon.tech keepalives=1 keepalives_idle=30 keepalives_interval=10 keepalives_count=5 sslmode=require', closed: 0>

In [1]:
import sys
import os

os.chdir("..")
from modules.data.download import download_data
download_data("2025-06-01", "2025-07-01")

/Users/jreid/Library/CloudStorage/Dropbox/programming/Projects/common_place/backend/src/modules/utils/../../../.env
environ({'COMMAND_MODE': 'unix2003', 'HOME': '/Users/jreid', 'LOGNAME': 'jreid', 'MallocNanoZone': '0', 'NVM_BIN': '/Users/jreid/.nvm/versions/node/v24.5.0/bin', 'NVM_CD_FLAGS': '-q', 'NVM_DIR': '/Users/jreid/.nvm', 'NVM_INC': '/Users/jreid/.nvm/versions/node/v24.5.0/include/node', 'OLDPWD': '/', 'ORIGINAL_XDG_CURRENT_DESKTOP': 'undefined', 'PATH': '/Users/jreid/Library/CloudStorage/Dropbox/programming/Projects/common_place/.venv/bin:/Users/jreid/.nvm/versions/node/v24.5.0/bin:/Library/Frameworks/Python.framework/Versions/3.13/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin', 'PWD': '/', 'SHELL': '/bin/zsh', 'SHLVL': '1',

2025-08-20 16:22:28 [INFO] Members and parties data not found in the database. Downloading...


Fetching members with skip=0
Fetching members with skip=1000
Fetching members with skip=2000
Fetching members with skip=3000
Fetching members with skip=4000
Fetching members with skip=5000


2025-08-20 16:27:23 [INFO] Downloaded 5191 members and 48 parties.


Found 915 debates.
Processing debate 1/915: 51CF381C-7768-4230-BEF1-E7C1B097561B
Processing debate 51/915: 3fad4c06-2564-4db4-bf77-cda70eb0897f
Processing debate 101/915: a988a68c-7501-4f2e-ada3-f798dc2dae4d
Processing debate 151/915: 2E1FAF27-1DFA-4675-8171-28F2FFB2BE13
Processing debate 201/915: DE97FFF9-2087-4476-A71B-663227CC3E62
Processing debate 251/915: E69E5711-DBC3-4721-A686-1699A7FB79DD
Processing debate 301/915: ED21BEF6-7741-413D-A595-23064EDAD4D2
Processing debate 351/915: A676AA06-6199-4DB7-BF0B-8923EFF9DF75
Processing debate 401/915: 8948F295-9B8D-4F99-A9E5-B3BE1B44E4AF
Processing debate 451/915: 75A436AD-5E70-4320-8947-53FBBBCCDF69
Processing debate 501/915: 7CEE14EB-BC5B-427F-A3A8-C3F0D8680AAF
Processing debate 551/915: 47db2216-928f-4cce-95be-5baccdf253bd
Processing debate 601/915: 1C6F3A95-74B5-4B8D-8D6C-6B64AD91A12D
Processing debate 651/915: a0fb0e1a-0040-452e-9df1-32b7ce061a4c
Processing debate 701/915: 91256B35-781A-4900-B267-3B5C41AB5C81
Processing debate 751/91

2025-08-20 16:31:56 [INFO] Downloaded 915 debates and 33518 contributions.
2025-08-20 16:35:46 [INFO] Debates and contributions data downloaded successfully.


In [1]:
import sys
import os

from modules.points.generate import generate_points
generate_points(batch_size=200, filters={"start_date": "2025-06-01", "end_date": "2025-07-01"})

Analysed: 317 Unanalysed: 125
Starting analysis pass 1 with batch size 200
Found 125 unanalysed debates. Processing...
Processing debate 1/125: Topical Questions (ID: C4DEBF3D-B21D-413A-8A41-18ED3646836A)


KeyboardInterrupt: 

In [2]:
import numpy as np
from psycopg2.extras import execute_values

def to_fp16_bytes(x32, dims=256):
    return np.asarray(x32, dtype=np.float32)[:dims].astype(np.float16).tobytes()

def backfill_emb256_f16(conn, dims=256, batch=5000):
    with conn, conn.cursor(name="full_vectors") as rcur:
        rcur.itersize = batch
        rcur.execute("""
            SELECT point_id, (point_embedding::real[]) AS emb
            FROM point
            WHERE point_embedding IS NOT NULL AND emb256_f16 IS NULL
        """)
        buf = []
        with conn.cursor() as wcur:
            for rows in iter(lambda: rcur.fetchmany(batch), []):
                for pid, emb in rows:
                    buf.append((pid, to_fp16_bytes(emb, dims)))
                execute_values(
                    wcur,
                    """
                    UPDATE point AS p
                    SET emb256_f16 = data.emb
                    FROM (VALUES %s) AS data(point_id, emb)
                    WHERE p.point_id = data.point_id
                    """,
                    buf,
                    template="(%s,%s)"
                )
                buf.clear()


In [3]:
backfill_emb256_f16(get_db_connection())

In [ ]:
import psycopg2
from typing import Dict
import base64, json

def get_db_connection():
    return psycopg2.connect(
        host="localhost",
        database="parliament",
        user="parliament_user",
        password="parliament_pass",
        port=5432
    )

def create_cluster_jobs_and_clusters_tables():
    conn = get_db_connection()
    cur = conn.cursor()
    # Create cluster_jobs table
    cur.execute("""
    CREATE TABLE IF NOT EXISTS cluster_jobs (
      job_id        BIGSERIAL PRIMARY KEY,
      status        TEXT NOT NULL CHECK (status IN ('queued','running','complete','failed','canceled')),
      root_cluster_id INTEGER REFERENCES clusters(cluster_id) ON DELETE SET NULL,
      params        JSONB NOT NULL,
      created_at    TIMESTAMPTZ NOT NULL DEFAULT now(),
      started_at    TIMESTAMPTZ,
      finished_at   TIMESTAMPTZ,
      error         TEXT
    );
    """)
    # Add job_id and visible columns to clusters table if not exist
    cur.execute("""
    ALTER TABLE clusters
    ADD COLUMN IF NOT EXISTS job_id BIGINT REFERENCES cluster_jobs(job_id);
    """)
    cur.execute("""
    ALTER TABLE clusters
    ADD COLUMN IF NOT EXISTS visible BOOLEAN NOT NULL DEFAULT FALSE;
    """)
    conn.commit()
    cur.close()
    conn.close()

create_cluster_jobs_and_clusters_tables()